# 1. Insert Boolean logic text
### (1) Spacing is required between parentheses, names, and logic operators.
    
    ex) "A = ( B and ( not C or D ) ) and E"

### (2) Set the state ("True" or "False") of the input nodes.

    ex) "A = False"

In [15]:
# Boolean network example in figure 4
booleanlogic = '''
A = D
B = not A
C = A and not B
D = C
E = C or F or G
F = not E
G = G or F
H = D or I
I = F or H
'''

# 2. Preprocessing Boolean logic text
### change node names into number nodes
(Please check booleanlogicNum before performing the following steps to avoid possible errors.)

In [16]:
from dcgs import booleanlogic_preprocessing

booleanlogicNum, node2num, num2node = booleanlogic_preprocessing.modeltext2nummodeltext(booleanlogic)
dgraph, nodeList, inputNodeState = booleanlogic_preprocessing.get_interaction_network(booleanlogicNum)
print(booleanlogicNum)


n1 = n4
n2 = not n1
n3 = n1 and not n2
n4 = n3
n5 = n3 or n6 or n7
n6 = not n5
n7 = n7 or n6
n8 = n4 or n9
n9 = n6 or n8



# 3. Get minimal feedback vertex sets (FVSs)
### get FVSs of each strongly connected component and combine them.

In [25]:
from dcgs import sccTofvs
dic_hierarchy, dic_fvs, minimal_fvs = sccTofvs.scc2fvs_bruteforce(dgraph)
print('FVS of each SCC: ',dic_fvs)
print('FVS example: ',minimal_fvs)

FVS of each SCC:  {'n9_n8': [['n9'], ['n8']], 'n5_n6_n7': [['n5', 'n7'], ['n6', 'n7']], 'n2_n4_n3_n1': [['n4'], ['n3'], ['n1']]}
FVS example:  ['n4', 'n5', 'n7', 'n9']


# 4. Get point attractors
### get point attractors from canalized state set of FVS

In [18]:
from dcgs import attractorlandscapeSeeker
CSS, state_origin = attractorlandscapeSeeker.get_canalized_states(booleanlogicNum, dgraph, nodeList, inputNodeState, [minimal_fvs])
paStr,paDic = attractorlandscapeSeeker.get_pointattractorFromCSS(booleanlogicNum, CSS)
print('Point attractors: ',paStr)

Point attractors:  ['010010100', '010010111', '101110011', '101110111']


# 5. Get control sets for global stabilization using DCGS
### get control sets (canalizing sets) for global stabilization of a target attractor

In [27]:
from dcgs import Main
targetattractor = paStr[3]
dic_CS,dic_cCS, dic_fvs = Main.algorithm(booleanlogicNum, targetattractor)
print('control sets (canalizing sets) for each FVS of each SCC: ',dic_CS)

control sets (canalizing sets) for each FVS of each SCC:  {'n2_n4_n3_n1': [['n4'], ['n3'], ['n1'], ['n4'], ['n3'], ['n1'], ['n4'], ['n3'], ['n1'], ['n4'], ['n3'], ['n1']], 'n5_n6_n7': [['n7'], ['n7'], ['n7'], ['n7'], ['n7'], ['n7'], ['n7'], ['n7'], ['n7'], ['n7'], ['n7'], ['n7']], 'n9_n8': [[]]}


### aggregate canalizing sets of each SCC

In [ ]:
import copy
controlSet = []
for scc in dic_CS:
    csList = dic_CS[scc]
    if csList == [[]]:
        continue
    if controlSet == []:
        controlSet = csList
        continue
    controlSet_copy = copy.deepcopy(controlSet)
    for cs_origin in controlSet:
        for cs in csList:
            controlSet_copy.append()

# 6. Compare with the result from the brute forece CK algorithm